In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
import sys

path = os.path.abspath("/kaggle/input/datatesting")
sys.path.append(path)
path = os.path.abspath("/kaggle/input/ocular-disease-recognition-odir5k")

sys.path.append(path)

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

## Import Modules and Libraries 


In [ ]:
import os
import sys
import copy
import time
import argparse
import importlib
import numpy as np
from torch.utils.data import DataLoader
import random
from keras.metrics import Accuracy, F1Score,AUC
from sklearn.metrics import roc_curve, auc, average_precision_score
from torchvision import transforms
import torchvision
from tensorflow.keras.metrics import AUC
from torchvision.models import resnet50
import gc
from  sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
label2disease = ['N', 'D', 'G', 'C', 'A', 'H', 'M']
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np


## Read the Data 

In [ ]:
csv_path = "/kaggle/input/finaaaaaal-data/final_processed_modified_lastV_inShaaAllah.csv"
images_folder="/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images"
df = pd.read_csv(csv_path)

# Testing Some Labels 


## Shuffle and Split the Data

In [ ]:
df=df.sample(len(df),random_state=7)


In [ ]:
print(df.shape)



In [ ]:
df=df[df['O']!=1]

In [ ]:
labels_frequancy=[0,0,0,0,0,0,0,0]
for index, row in df.iterrows():
    for i,diease in enumerate(label2disease):
        if row[diease]==1:
            labels_frequancy[i]+=1 

In [ ]:
print(labels_frequancy)

## Filter data to get just required classes 


In [ ]:
others_class =df[df['target']=="[0, 0, 0, 0, 0, 0, 0, 1]"]

In [ ]:
print(others_class.shape)

In [ ]:
hypertension_class =df[df['target']=="[0, 0, 0, 0, 0, 1, 0, 0]"]

In [ ]:
print(hypertension_class.sample(5))

In [ ]:
glacouma_class =df[df['target']=="[0, 0, 1, 0, 0, 0, 0, 0]"]

In [ ]:
cataract_class =df[df['target']=="[0, 0, 0, 1, 0, 0, 0, 0]"]

In [ ]:
amd_class =df[df['target']=="[0, 0, 0, 0, 1, 0, 0, 0]"]

In [ ]:
diabetic_class =df[df['target']=="[0, 1, 0, 0, 0, 0, 0, 0]"]

In [ ]:
print(others_class.shape,hypertension_class.shape,glacouma_class.shape,amd_class.shape)

In [ ]:
aug_classes =pd.DataFrame()

In [ ]:
aug_classes=pd.concat([aug_classes, hypertension_class], ignore_index=True)

In [ ]:
aug_classes=pd.concat([aug_classes, glacouma_class], ignore_index=True)

In [ ]:
aug_classes=pd.concat([aug_classes, amd_class], ignore_index=True)

In [ ]:

aug_classes=pd.concat([aug_classes, cataract_class], ignore_index=True)

In [ ]:

aug_classes=pd.concat([aug_classes, diabetic_class], ignore_index=True)

In [ ]:
aug_classes=aug_classes.sample(len(aug_classes))

In [ ]:
print(aug_classes)

In [ ]:
print(df.shape[0]+aug_classes.shape[0])

In [ ]:
print(label2disease)

In [ ]:
labels_frequancy=[0,0,0,0,0,0,0]
for index, row in df.iterrows():
    for i,diease in enumerate(label2disease):
        if row[diease]==1:
            labels_frequancy[i]+=1 

In [ ]:
print(labels_frequancy)

In [ ]:
common_rows = pd.merge(df, aug_classes, how='inner')

In [ ]:
print(common_rows)

In [ ]:
df = df[~df.apply(tuple, axis=1).isin(common_rows.apply(tuple, axis=1))]

In [ ]:
common_rows = pd.merge(df, aug_classes, how='inner')


In [ ]:
print(common_rows)

In [ ]:
label_counts = df[label2disease].value_counts()

In [ ]:
print(label_counts)

In [ ]:
from sklearn.model_selection import train_test_split

x_train , x_val , y_train, y_val = train_test_split(df["image"],df[label2disease], test_size=0.2,random_state=9)

# x_test , x_val , y_test, y_val = train_test_split(x_temp,y_temp, test_size=0.5)


In [ ]:
train_data= pd.concat([x_train, y_train], axis = 1) 
# test_data= pd.concat([x_test, y_test], axis = 1) 
val_data= pd.concat([x_val, y_val], axis = 1) 

In [ ]:
print(train_data.shape[0]+val_data.shape[0]+aug_classes.shape[0])

In [ ]:
labels_taining_frequancy=[0,0,0,0,0,0,0]
for index, row in train_data.iterrows():
    for i,diease in enumerate(label2disease):
        if row[diease]==1:
            labels_taining_frequancy[i]+=1 

In [ ]:
print(type(train_data))

In [ ]:
print(labels_taining_frequancy)

# balancing Validation Data by shuffiling and concatinating 

In [ ]:
conditions=["[0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0]"]
def balance_validation_data(train_df, val_df, label_columns):
       # Copying rows that meet the condition to df2
    for condition in conditions :
        print(train_df.sample(5))
        sampled_rows = train_df[train_df['target']==condition].sample(20)  # Change the sample size as needed
        print(sampled_rows.shape)
        val_df = pd.concat([val_df, sampled_rows], ignore_index=True)
        # Removing the moved rows from df1
        train_df = train_df.drop(sampled_rows.index)

    return train_df, val_df



In [ ]:
aug_classes,val_data=balance_validation_data(aug_classes,val_data,label2disease)

In [ ]:
labels_val_frequancy=[0,0,0,0,0,0,0]
for index, row in val_data.iterrows():
    for i,diease in enumerate(label2disease):
        if row[diease]==1:
            labels_val_frequancy[i]+=1 

In [ ]:
print(aug_classes.shape,train_data.shape,val_data.shape)

In [ ]:
print(labels_taining_frequancy,labels_val_frequancy)

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight


In [ ]:
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

In [ ]:
print(sample_weights[0:10])
print(y_train[0:10])

## Read the Images and Assign Labels 

### Apply Preprocessing 

In [ ]:
from PIL import Image
def loadImg(path):

    img = Image.open(path)
    img = img.convert("RGB")
    img_array = np.array(img)
    
    return img_array

def cropImg(image , percentage = 0.8):
    height, width, _ = image.shape
    new_height = int(height * percentage)
    new_width = int(width * percentage)

    start_x = (width - new_width) // 2
    start_y = (height - new_height) // 2

    centered_region = image[start_y:start_y+new_height, start_x:start_x+new_width]
    
    return centered_region

def resizeImg(image, target_width=512, target_height =512):

    original_height, original_width = image.shape[:2]
    aspect_ratio = original_width / original_height
    target_aspect_ratio = target_width / target_height

    if aspect_ratio > target_aspect_ratio:
        new_width = target_width
        new_height = int(target_width / aspect_ratio)
    else:
        new_width = int(target_height * aspect_ratio)
        new_height = target_height

    resized_image = cv.resize(image, (new_width, new_height), interpolation=cv.INTER_CUBIC)
    canvas = 255 * np.ones((target_height, target_width, 3), dtype=np.uint8)

    x_offset = (target_width - new_width) // 2
    y_offset = (target_height - new_height) // 2

    canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image

    return canvas

def crop(image):

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (5, 5), 0)

    # Detect circles using Hough Circle Transform
    circles = cv.HoughCircles(blurred, cv.HOUGH_GRADIENT, dp=1, minDist=20,
                               param1=50, param2=30, minRadius=0, maxRadius=0)

    if circles is not None:
        # Convert the (x, y) coordinates and radius of the circle to integers
        circles = np.round(circles[0, :]).astype("int")

        # Assuming there's only one circle, extract its coordinates and radius
        (x, y, r) = circles[0]

        # Create a mask for the circle
        mask = np.zeros_like(gray)
        cv.circle(mask, (x, y), r, 255, thickness=-1)

        # Crop the circle from the original image using the mask
        masked_image = cv.bitwise_and(image, image, mask=mask)
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        max_contour = max(contours, key=cv.contourArea)
        x, y, w, h = cv.boundingRect(max_contour)
        cropped_image = masked_image[y:y+h, x:x+w]
        resized_image = cv.resize(cropped_image,(224,224))
        return resized_image , True


    else:
        print("No circles detected.")
        return image , False
    
def rotate_image(image,angle):
    # Get the image size
    (height, width) = image.shape[:2]

    # Define the center of the image
    center = (width / 2, height / 2)
    scale = 1.0
    matrix = cv.getRotationMatrix2D(center, angle, scale)
    rotated_by_matrix = cv.warpAffine(image, matrix, (width, height))
    image_bgr=rotated_by_matrix
    return image_bgr;

def gamma_correction(image):
   # Gamma Correction
    gamma = 1.2
    lookup_table = np.array([((i / 255.0) ** gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    gamma_corrected_image = cv.LUT(image, lookup_table)
    return gamma_corrected_image ;


def flip(image,direction=0):
    image = cv.flip(image, direction) 
    return image 

def fill(img, h, w):
    img = cv.resize(img, (h, w), cv.INTER_CUBIC)
    return img
        
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img


def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = fill(img, h, w)
    return img


def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, h, w)
    return img

def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img = np.clip(img, 0, 255)  # Clip values to stay within [0, 255]
    img = img.astype(np.uint8)
    return img


In [ ]:
def enhanceImg(img):
    conv_img = cv.cvtColor(img,  cv.COLOR_RGB2YCrCb)
    Y_channel = conv_img[: , : ,0]
    Y_channel = Y_channel.astype(np.uint8)
    
    clahe = cv.createCLAHE(clipLimit=10.0, tileGridSize=(16,16))
    cl = clahe.apply(Y_channel)
    
    merged = cv.merge((cl , conv_img[:,:,1], conv_img[:,:,2]))
    modified_img = cv.cvtColor(merged , cv.COLOR_YCrCb2RGB)
    
    return modified_img

In [ ]:
# imgs = df['iamge']
random.seed(42)
image_size=224
def dataset_generator_aug(dff):
    dataset = []
    for index, row in dff.iterrows():
        random.shuffle(dataset);
        img_path = os.path.join("/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images",row['image'])
        try:
            # Load image in color mode
            image = loadImg(img_path)
            image = resizeImg(image)
            image,f = crop(image)

            if f :
                labels=[0,0,0,0,0,0,0]
                for i,diease in enumerate(label2disease):
                    if row[diease]==1:
                        labels[i]=1 ;
                dataset.append([np.array(enhanceImg(image)),np.array(labels)])
            else:
                  print(f"Warning: Failed to crop image: {img_path}")
        except Exception as e:
            print(f"Error loading image: {img_path}: {e}")
    return dataset

In [ ]:
train_data=dataset_generator_aug(train_data)
# test_data=dataset_generator_aug(test_data)
val_data=dataset_generator_aug(val_data)

## Visulize the Images 


In [ ]:
plt.figure(figsize=(12,7))
for i in range(10):
    sample = random.choice(range(len(train_data)))
    
    image = train_data[sample][0]
    print(image.shape)
    category = train_data[sample][1]

    plt.subplot(2,6,i+1)
    plt.imshow(image)
plt.tight_layout()    

## Make Augmentation on the data :


In [ ]:
def dataset_generator_with_augmentation(dff):
    dataset = []
    for index, row in dff.iterrows():
        random.shuffle(dataset);
        img_path = os.path.join("/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images",row['image'])
        try:
            # Load image in color mode
            original_image = loadImg(img_path)
            original_image = resizeImg(original_image)
            original_image,f = crop(original_image)
            image=enhanceImg(original_image) 
            if f :
                labels=[0,0,0,0,0,0,0]
                for i,diease in enumerate(label2disease):
                    if row[diease]==1:
                        labels[i]=1 ;
                        # 5 5 7 12 6
                        
                        
                if labels[1]==1:
                    dataset.append([np.array(channel_shift(image,int(random.uniform(10, 20)))),np.array(labels)])
                    dataset.append([np.array(flip(image,1)),np.array(labels)])
                    dataset.append([np.array(rotate_image(image,int(random.uniform(30, 150)))),np.array(labels)])
                    dataset.append([np.array(vertical_shift(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(zoom(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(original_image),np.array(labels)])
                if(labels[3]==1):
                    dataset.append([np.array(rotate_image(image,int(random.uniform(30, 120)))),np.array(labels)])
                    dataset.append([np.array(flip(image,1)),np.array(labels)])
                    dataset.append([np.array(vertical_shift(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(zoom(image,random.uniform(.1, .2))),np.array(labels)])

                if(labels[2]==1):
                    dataset.append([np.array(channel_shift(image,int(random.uniform(10, 20)))),np.array(labels)])
                    dataset.append([np.array(flip(image,1)),np.array(labels)])
                    dataset.append([np.array(rotate_image(image,int(random.uniform(30, 120)))),np.array(labels)])
                    dataset.append([np.array(zoom(image,random.uniform(.1, .2))),np.array(labels)])
                if labels[4]==1:
                    dataset.append([np.array(channel_shift(image,int(random.uniform(10, 20)))),np.array(labels)])
                    dataset.append([np.array(flip(image,1)),np.array(labels)])
                    dataset.append([np.array(rotate_image(image,int(random.uniform(30, 150)))),np.array(labels)])
                    dataset.append([np.array(vertical_shift(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(zoom(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(original_image),np.array(labels)])
                if labels[5]==1:
                    dataset.append([np.array(channel_shift(image,int(random.uniform(10, 20)))),np.array(labels)])
                    dataset.append([np.array(flip(image,1)),np.array(labels)])
                    dataset.append([np.array(rotate_image(image,int(random.uniform(30, 150)))),np.array(labels)])
                    dataset.append([np.array(vertical_shift(image,random.uniform(.1,.3))),np.array(labels)])
                    dataset.append([np.array(horizontal_shift(image,random.uniform(0, .3))),np.array(labels)])
                    dataset.append([np.array(gamma_correction(image)),np.array(labels)])
                    dataset.append([np.array(original_image),np.array(labels)])
                    dataset.append([np.array(zoom(image,random.uniform(.1,.25))),np.array(labels)])
                if labels[6]==1:
                    dataset.append([np.array(channel_shift(image,int(random.uniform(10, 20)))),np.array(labels)])
                    dataset.append([np.array(flip(image,1)),np.array(labels)])
                    dataset.append([np.array(rotate_image(image,int(random.uniform(30, 150)))),np.array(labels)])
                    dataset.append([np.array(vertical_shift(image,random.uniform(.1,.3))),np.array(labels)])
                    dataset.append([np.array(horizontal_shift(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(gamma_correction(image)),np.array(labels)])
                    dataset.append([np.array(original_image),np.array(labels)])
                    
                if labels[7]==1:
                    dataset.append([np.array(horizontal_shift(image,random.uniform(.1, .3))),np.array(labels)])
                    dataset.append([np.array(gamma_correction(image)),np.array(labels)])
                    dataset.append([np.array(original_image),np.array(labels)])
                    dataset.append([np.array(zoom(image,random.uniform(.1,.25))),np.array(labels)])

            else:
                  print(f"Warning: Failed to crop image: {img_path}")
        except Exception as e:
            print(f"Error loading image: {img_path}: {e}")
    return dataset

In [ ]:
augmanted_data=dataset_generator_with_augmentation(aug_classes)

In [ ]:
print(len(augmanted_data),len(train_data))

In [ ]:
for item in augmanted_data:
    train_data.append(item)

In [ ]:
gc.collect()

In [ ]:
random.seed(7)

In [ ]:
random.shuffle(train_data)

## Load the Data 


In [ ]:
#Divide the dataset into 2 predictors and target...
#here predictors would be the image and target would be the label
from tensorflow.keras.utils import to_categorical

train_x = np.array([i[0]/255.0 for i in train_data]).reshape(-1,image_size,image_size,3).astype(np.float32);
train_y = np.array([i[1]/1.0 for i in train_data]).astype(np.float32);

# test_x = np.array([i[0]/255.0 for i in test_data]).reshape(-1,image_size,image_size,3).astype(np.float32);
# test_y = np.array([i[1] for i in test_data])

val_x = np.array([i[0]/255.0 for i in val_data]).reshape(-1,image_size,image_size,3).astype(np.float32);
val_y = np.array([i[1]/1.0 for i in val_data]).astype(np.float32);



In [ ]:
print(train_y.shape)

## Visulize after Loaded 


In [ ]:
plt.figure(figsize=(12,7))
for i in range(10):
    sample = random.choice(range(len(train_x)))
    
    image = train_x[sample]
    print(image.shape)
    plt.subplot(2,6,i+1)
    plt.imshow(image)
plt.tight_layout()    

In [ ]:
plt.figure(figsize=(12,7))
for i in range(10):
    sample = random.choice(range(len(train_x)))
    
    image = train_x[sample]
    print(image.shape)
    plt.subplot(2,6,i+1)
    plt.imshow(image)
plt.tight_layout()    

In [ ]:
img_path = os.path.join("/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images","968_left.jpg")

# Load image in color mode
original_image = loadImg(img_path)
original_image = resizeImg(original_image)
original_image,f = crop(original_image)
image1=enhanceImg(original_image) 
image2=channel_shift(image1,40)

plt.figure(figsize=(12,7))
plt.subplot(1,2,1)
plt.imshow(image1)

plt.subplot(1,2,2)

plt.imshow(image2)


## Import Layers and Libraries 

In [ ]:
print(train_y.shape)

In [ ]:
freq_map={}
for image in train_y:
    if str(image) in freq_map:
        freq_map[str(image)]+=1 
    else:
        freq_map[str(image)]=1 
print(freq_map)

In [ ]:
freq_map={}
for image in val_y:
    if str(image) in freq_map:
        freq_map[str(image)]+=1 
    else:
        freq_map[str(image)]=1 
print(freq_map)

In [ ]:
labels_val_frequancy=[0,0,0,0,0,0,0]
for image in train_y:
    for i,dieases in enumerate(image) :
            if(dieases==1):
                labels_val_frequancy[i]+=1 

In [ ]:
print(labels_val_frequancy)

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate,Average,BatchNormalization,Dropout,Add,Multiply
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam,SGD,AdamW
from keras.losses import CategoricalCrossentropy,BinaryCrossentropy,BinaryFocalCrossentropy
from keras.optimizers import Adam
from keras.initializers import LecunUniform
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.applications import ConvNeXtBase ,EfficientNetB3,InceptionResNetV2,VGG19
import torch.nn.functional as F
from keras.regularizers import l2
from keras_cv.layers import ChannelShuffle
from keras import activations

In [ ]:
def DckNetLayer(x):
    num_filters = x.shape[-1]
    branch1 = Sequential()
    branch1.add(Conv2D(filters=num_filters, kernel_size=(2, 2), padding='same',
                                            activation='relu', dilation_rate=4,kernel_initializer="he_normal"))
    branch1.add(BatchNormalization())

    
    
    
    branch2 = Sequential()
    branch2.add(Conv2D(filters=num_filters, kernel_size=(2, 2), padding='same',
                                            activation='relu', dilation_rate=3,kernel_initializer="he_normal"))
    branch2.add(BatchNormalization())

    
    
    
    
    branch3 = Sequential()
    branch3.add(Conv2D(filters=num_filters, kernel_size=(2, 2), padding='same',
                                            activation='relu', dilation_rate=2,kernel_initializer="he_normal"))
    branch3.add(BatchNormalization())
    
    
    P3=branch1(x)
    P2=branch2(x)
    P1=branch3(x)
    
    
    MergedBranch1=Add()([P3, P2, P1])
    
    GMP=GlobalMaxPooling2D()(MergedBranch1)
    GAP=GlobalAveragePooling2D()(MergedBranch1)
    MergedBranch2=Add()([GMP, GAP])
    Q_dash=Dropout(0.5)(MergedBranch2)
    FC_layer = Sequential()
    FC_layer.add(Dense(num_filters, activation='relu',kernel_initializer="he_normal"))
    FC_layer.add(Dense(num_filters, activation='sigmoid',kernel_initializer="he_normal"))
    R=FC_layer(Q_dash)
    
    S3= Multiply()([P3,R])
    S2= Multiply()([P2,R])
    S1= Multiply()([P1,R])
    
    MergedBranch3=Add()([S3, S2, S1])
    
    output=MergedBranch3
    return output

In [ ]:
def SqueezeAndExcuetionLayer(x):
    num_filters = x.shape[-1]
    GAP1=GlobalAveragePooling2D()(x)
    FC_layer = Sequential()
    FC_layer.add(Dense(num_filters, activation='relu',kernel_initializer="he_normal"))
    FC_layer.add(Dense(num_filters, activation='sigmoid',kernel_initializer="he_normal"))
    FC1_output=FC_layer(GAP1)
    SAQ_output= Multiply()([FC1_output,x])
    return SAQ_output
    

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
print(train_y.dtype)


In [ ]:


model_pre1 = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(image_size,image_size,3))
model_pre1.trainable=False ;

for layer in model_pre1.layers[:20]:
    print(layer.trainable)     
# Define the model creation function
def create_multimodal_model(image_size, l2_reg=0.001):
    img1_input = Input(shape=(image_size, image_size, 3))
    features = model_pre1(img1_input)
    print(features)
    features_shape=features.shape[-1]
    x=ChannelShuffle(groups=3,seed=17)(features)
    x=DckNetLayer(x)
    x=SqueezeAndExcuetionLayer(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(1024,kernel_initializer="he_normal")(x)
    x=activations.relu(x)
    x=Dropout(.5)(x)
    x = BatchNormalization()(x)
    x = Dense(1024,kernel_initializer="he_normal")(x)
    x=activations.relu(x)
    x=Dropout(.35)(x)
    x = BatchNormalization()(x)
    x = Dense(7,kernel_initializer="he_normal",kernel_regularizer=l2(0.01))(x)
    x=activations.sigmoid(x)
    output = x

    return Model(inputs=img1_input, outputs=output)

# Compile the model
model = create_multimodal_model(224)
model.summary()


In [ ]:


# model_pre1 = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(image_size,image_size,3))
# model_pre1.trainable=False ;

# for layer in model_pre1.layers[:20]:
#     print(layer.trainable)     
# # Define the model creation function
# def create_multimodal_model(image_size, l2_reg=0.001):
#     img1_input = Input(shape=(image_size, image_size, 3))
#     features = model_pre1(img1_input)
#     shuffled=ChannelShuffle(groups=3,seed=42)(features)
#     x=DckNetLayer(shuffled)
#     x=SqueezeAndExcuetionLayer(x)
#     x = GlobalAveragePooling2D()(x)
#     x = Dropout(0.5)(x)
#     x = Dense(256, activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = Dense(8, activation='sigmoid')(x)    
#     output = x

#     return Model(inputs=img1_input, outputs=output)

# # Compile the model
# model = create_multimodal_model(224)
# model.summary()


In [ ]:
print(model.layers[1].trainable)

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_schedule=0.0005

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","f1_score","auc"])

es_callback = EarlyStopping(patience=30,monitor='val_accuracy',mode='max', verbose=1,restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=7, verbose=1)


In [ ]:
print(train_x.shape,train_y.shape)


In [ ]:
sample_weights = compute_sample_weight(class_weight='balanced', y=train_y)

In [ ]:
# model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy",AUC()])
# class_weights={2: 0.3,4:0.8,5:0.8,7:0.3}
# history =model.fit(train_x, train_y, 
#           validation_data=(val_x, val_y),
#           epochs=55,callbacks=[es_callback, reduce_lr], batch_size=32,class_weight=class_weights)
history =model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          epochs=50,callbacks=[es_callback, reduce_lr], batch_size=32,sample_weight=sample_weights)

In [ ]:
model.trainable = True
inception_resnet_v2_block = None
sequintail=[]
for layer in model.layers:
    if layer.name[0:-1]=="batch_normalization":
        layer.trainable=False
    else:
        layer.trainable=True
        
    if layer.name == 'inception_resnet_v2':
        inception_resnet_v2_block = layer
    if(layer.name[0:10]=="sequential"):
        sequintail.append(layer)


for layer in inception_resnet_v2_block.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable=False
        
        
# for seq in sequintail:
#     for layer in seq.layers :
#         if layer.name[0:19]=="batch_normalization":
#             layer.trainable=False
#         else:
#             layer.trainable=True

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_schedule=.01

model.compile(optimizer=Adam(0.00005), loss="binary_crossentropy", metrics=["accuracy","f1_score","auc"])

es_callback = EarlyStopping(patience=10,monitor='val_accuracy',mode='max',verbose=1,restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, verbose=1)


In [ ]:
# model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy",AUC()])
print(train_x.shape,train_y.shape)

history =model.fit(train_x, train_y, 
          validation_data=(val_x, val_y),
          epochs=100,callbacks=[es_callback, reduce_lr], batch_size=16,shuffle=True,sample_weight=sample_weights)

In [ ]:
loss,accuracy,f1score,auc = model.evaluate(val_x,val_y)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
predict_y=model.predict(val_x)
binary_predictions = (predict_y >= 0.5).astype(int)
print(classification_report(val_y, binary_predictions, target_names=label2disease))


In [ ]:
model.save("without_other_90%PrectModel.keras")

In [ ]:
from tensorflow.keras.saving import load_model

In [ ]:
model2= load_model("/kaggle/working/without_other_90%PrectModel.keras")

In [ ]:
import os
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request

from google_auth_oauthlib.flow import InstalledAppFlow

from google.oauth2.credentials import Credentials


SCOPES = ["https://www.googleapis.com/auth/drive"]

def upload_basic(filename,token_path):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "cred.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_path, "w") as token:
            token.write(creds.to_json())

    try:
        # create drive api client
        service = build("drive", "v3", credentials=creds)

        file_metadata = {"name": f"{filename}"}
        media = MediaFileUpload(f"{filename}")
        # pylint: disable=maybe-no-member
        file = (
            service.files()
            .create(body=file_metadata, media_body=media, fields="id")
            .execute()
        )
        print(f'File ID: {file.get("id")}')

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None





In [ ]:
upload_basic("/kaggle/working/without_other_90%PrectModel.keras","/kaggle/input/a5eermara/token.json")

In [ ]:
test_other=[]
for index, row in others_class.iterrows():
        img_path = os.path.join("/kaggle/input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images",row['image'])
        try:
            # Load image in color mode
            image = loadImg(img_path)
            image = resizeImg(image)
            image,f = crop(image)
            image=enhanceImg(original_image) 

            labels=[0,0,0,0,0,0,0]
            for i,diease in enumerate(label2disease):
                if row[diease]==1:
                    labels[i]=1 ;        
            test_other.append([np.array(image),np.array(labels)])
        except Exception as Ex :
            print(Ex)

In [ ]:
test_result_x = np.array([i[0]/255.0 for i in test_other]).reshape(-1,image_size,image_size,3).astype(np.float32);
test_result_y = np.array([i[1]/1.0 for i in test_other]).astype(np.float32);

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
predict_y=model.predict(test_result_x)
binary_predictions = (predict_y >= 0.35).astype(int)

In [ ]:
print(binary_predictions[0:2])

In [ ]:

wrongs=0 
for pred in binary_predictions:
    if (predastype(int) == [0,0,0,0,0,0,0]).all() :
        wrongs+=1 ;
print(wrongs)
print(wrongs/len(binary_predictions*1.0))
# print(classification_report(test_result_y, binary_predictions, target_names=label2disease))